In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# MLP
In this notebook we demonstrate how the catastrophic forgetting heavily affect MLPs.
We train an MLP on a sorted MNIST. In the evaluation phase, the MLPs respond with the last seen digits.

In [ ]:
class MLP(nn.Module):
    def __init__(self, checkpoint: str | None = None):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)
        if checkpoint:
            self.load_state_dict(torch.load(checkpoint))
            self.eval()

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
# Train set. Here we sort the MNIST by digits and disable data shuffling.
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
sorted_indices = sorted(range(len(train_dataset)), key=lambda idx: train_dataset.targets[idx])
train_dataset = torch.utils.data.Subset(train_dataset, sorted_indices)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=False)
# Test set
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [ ]:
# Training the model
def train(model, checkpoint, epochs=5):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    for epoch in range(epochs):
        for images, labels in train_loader:
            outputs = model(images)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        print(f'Epoch {epoch + 1}, Loss: {loss.item()}')
        torch.save(model.state_dict(), checkpoint)

In [ ]:
# Loading model from checkpoint
model = MLP(checkpoint='checkpoint/mlp_mnist_no_shuffle.pth')

In [ ]:
# Evaluating
correct = 0
total = 0
predictions = [0] * 10
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        for pred in predicted:
            predictions[pred] += 1
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {100 * correct / total}%')
print(predictions)
print(sum(predictions))

Accuracy: 10.29%
[0, 0, 1, 1, 0, 1, 466, 31, 1656, 7844]
10000
